In [1]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [2]:
!pip install -qq -U datasets transformers pyarrow torchinfo
!pip install -qq --upgrade transformers ftfy accelerate regex tqdm
!pip install git+https://github.com/openai/CLIP.git
!pip install --q -U pytorch-lightning lightning-bolts
!pip install --upgrade -q wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 918.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.0/774.0 kB 12.6 MB/s et

In [3]:
!git clone "https://github.com/jyanivaddi/dl_hub.git"
!git -C dl_hub pull
!git pull

Cloning into 'dl_hub'...
remote: Enumerating objects: 645, done.
remote: Counting objects: 100% (397/397), done.
remote: Compressing objects: 100% (207/207), done.
remote: Total 645 (delta 250), reused 291 (delta 183), pack-reused 248
Receiving objects: 100% (645/645), 184.64 KiB | 3.36 MiB/s, done.
Resolving deltas: 100% (394/394), done.
Already up to date.
fatal: not a git repository (or any of the parent directories): .git


In [4]:
import sys
sys.path.append("ERA_V1/session_13")
sys.path.append("dl_hub")

from dl_hub.multi_modal_phi.dataset import (
    fetch_captions_and_images,
    PreTrainDataset
)
from dl_hub.multi_modal_phi.model import LitMultiModalGPT
from dl_hub.multi_modal_phi.main import (
    PeriodicCheckpoint,
    PrintAccuracyAndLoss,
    train_multimodal_gpt_model
)

In [ ]:
del fetch_captions_and_images

NameError: name 'fetch_captions_and_images' is not defined

In [5]:
import os
import gc
import torch
import pickle
import json
import torchinfo
import torchmetrics
import wandb
import requests
import torch.nn as nn
import numpy as np
import pytorch_lightning as pl
from PIL import Image
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from tqdm import tqdm
from pathlib import Path
from numba import cuda
from typing import Union, List
import torch.multiprocessing as mp
from torch.cuda.amp import autocast
from matplotlib import pyplot as plt
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.callbacks import Callback
from transformers import AutoProcessor, CLIPVisionModel



**Define Configuration**

In [6]:
json_path = '/content/gdrive/MyDrive/captions_train2017.json'
batch_size = 1
device = 'cuda' if torch.cuda.is_available() else 'cpu'
projection_layer_in_channels = 50
projection_layer_out_channels = 2560
max_training_steps = 100000
seq_len = 50
log_dir = 'phi_pretrain'
exp_name = 'phi2_proj_layer'
check_point_save_dir = 'phi2_projection_checkpoints/'
os.makedirs(check_point_save_dir,exist_ok = True)
save_freq = 1000
val_check_interval = 1000
lr = 1e-4

In [7]:
# Instantiate WandB logger
wandb.login()
wandb_logger = WandbLogger(save_dir=log_dir, name='mm_phi_stage1')
text_table = wandb.Table(columns=["training_step", "loss", "text"])
val_data_log = []


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: gyani123. Use `wandb login --relogin` to force relogin
wandb: WARNING Path phi_pretrain/wandb/ wasn't writable, using system temp directory.


**Define models and tokenizers**

In [8]:
# Define the models and tokenizers
clip_model = CLIPVisionModel.from_pretrained("openai/clip-vit-base-patch32")
clip_preprocessor = AutoProcessor.from_pretrained("openai/clip-vit-base-patch32")
phi_tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
# Define dataset and dataloaders
captions, raw_images_list, image_ids_list = fetch_captions_and_images(json_path)
image_ids_list = np.array(image_ids_list)
rand_indices = np.arange(len(image_ids_list))
np.random.shuffle(rand_indices)
val_indices = rand_indices[:100]
train_indices = rand_indices[100:]
train_image_ids = image_ids_list[train_indices]
val_image_ids = image_ids_list[val_indices]

# lets keep val size to 100
print(f"Train dataset size: {len(train_indices)}")
print(f"Valid dataset size: {len(val_indices)}")




total image ids: 118287, total images: 118287, total captions: 118287
Train dataset size: 118187
Valid dataset size: 100


In [33]:
def get_image_embeddings(image_url, model, preprocessor, device=None):
    """
    This method computes the clip embeddings for a given image, after preprocessing it according to the model
    """
    image = Image.open(requests.get(image_url, stream=True).raw)

    processed_image = preprocessor(images=image, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**processed_image)
    return outputs.last_hidden_state.squeeze()

class PreTrainDataset(Dataset):

    def __init__(self,
                 image_ids,
                 raw_images_list,
                 captions,
                 tokenizer,
                 clip_model,
                 clip_preprocessor,
                 device,
                 seq_len=50):
        super().__init__()
        self.tokenizer = tokenizer
        self.ds = None
        self.clip_model = clip_model
        self.clip_preprocessor = clip_preprocessor
        self.bos_token = self.tokenizer.bos_token
        self.eos_token = self.tokenizer.eos_token
        self.eos_token_id = 50256
        self.COMMENT_TOKEN_ID = 23893
        self.seq_len = seq_len
        self.captions = captions
        self.raw_images_list = raw_images_list
        self.image_ids = image_ids


    def __len__(self):
        return len(self.image_ids)


    def __getitem__(self, idx):

        # get image embeddings
        this_img_id = "{:012d}".format(int(self.image_ids[idx]))
        image_embeddings = get_image_embeddings(self.raw_images_list[self.image_ids[idx]], clip_model, clip_preprocessor)

        # get caption
        caption = self.captions[int(this_img_id)]
        tokenized_caption = self.tokenize_caption(caption)

        return {
            "image_embeddings": image_embeddings.unsqueeze(0),
            "caption": caption,
            "tokenized_caption": tokenized_caption,
            "token_len": len(tokenized_caption)
        }

    def tokenize_caption(self, caption):
        tokenizer_output = self.tokenizer(caption, return_tensors="pt", return_attention_mask=False)
        tokenized_caption = tokenizer_output['input_ids'].squeeze()
        #if len(caption_encoded) > self.seq_len:
        #    caption_encoded = caption_encoded[:self.seq_len-2]
        #num_padding_tokens = self.seq_len - len(caption_encoded) + 1
        ## Add <s> and </s> token
        #tokenized_caption = torch.cat(
        #    [
        #        caption_encoded.squeeze(),
        #        torch.tensor([self.tokenizer.eos_token_id]*num_padding_tokens,dtype=torch.int64),
        #    ],dim=0)
        ##print(f"caption length: {len(caption_encoded)} number of padding tokens: {num_padding_tokens} total size: {len(tokenized_caption)}")
        return tokenized_caption


    def collate_samples(self, batch):
        """
        Perform dynamic batching on the sequences.
        For each batch, we get the length of the longest sentence and pad the remaining sentences according to that.
        """

        #print("inside collate function")
        # max encoder str length
        max_len = max(x["token_len"] for x in batch)
        print(f"longest token in this batch: {max_len}")

        captions_list = []
        image_embeddings_list = []
        tokenized_captions_list = []

        for cnt, x in enumerate(batch):
            # Add sos, eos and padding to each sentence
            num_padding_tokens = max(0, max_len - len(x["tokenized_caption"]))+1  # we will add <s> and </s>

            # Add <s> and </s> token
            batch_x = torch.cat(
                [
                    x['tokenized_caption'],
                    torch.tensor([self.eos_token_id] * num_padding_tokens, dtype=torch.int64),
                ],
                dim=0,
            )
            tokenized_captions_list.append(batch_x)
            image_embeddings_list.append(x['image_embeddings'])
            captions_list.append(x['caption'])

        #print("inside get item and I am returning the dict list!")
        return {
            "image_embeddings": torch.vstack(image_embeddings_list),
            "tokenized_captions": torch.vstack(tokenized_captions_list).unsqueeze(1),
            "captions": captions_list,
        }

In [34]:
train_ds = PreTrainDataset(train_image_ids,
                            raw_images_list,
                            captions,
                            phi_tokenizer,
                            clip_model,
                            clip_preprocessor,
                            device,
                            seq_len=seq_len)

val_ds = PreTrainDataset(val_image_ids,
                            raw_images_list,
                            captions,
                            phi_tokenizer,
                            clip_model,
                            clip_preprocessor,
                            device,
                            seq_len=seq_len)

In [35]:
train_dataloader = DataLoader(dataset = train_ds,
                            batch_size = 2,
                            num_workers = 1,
                            collate_fn = train_ds.collate_samples,
                            shuffle = True)
val_dataloader = DataLoader(dataset = val_ds,
                            batch_size = 1,
                            num_workers = 1,
                            collate_fn = val_ds.collate_samples,
                            shuffle = True)


In [36]:
eg = next(iter(train_dataloader))

longest token in this batch: 11
longest token in this batch: 12


In [37]:
print(eg.keys())

dict_keys(['image_embeddings', 'tokenized_captions', 'captions'])


In [39]:
eg['image_embeddings'].shape
print(eg['tokenized_captions'].shape)

torch.Size([2, 1, 12])


In [ ]:
# Define multimodal model
multimodal_gpt_model = LitMultiModalGPT(projection_layer_in_channels,
                                        projection_layer_out_channels)
optimizer = torch.optim.Adam(multimodal_gpt_model.parameters(), lr=1.0e-4, eps=1e-9)
multimodal_gpt_model.set_optimizer(optimizer)

# define and train the model
trainer = train_multimodal_gpt_model(multimodal_gpt_model, train_dataloader, val_dataloader, logger = wandb_logger, max_training_steps=max_training_steps)


config.json:   0%|          | 0.00/863 [00:00<?, ?B/s]

configuration_phi.py:   0%|          | 0.00/9.26k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi.py:   0%|          | 0.00/62.7k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- modeling_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]